In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.linear_model import LinearRegression, Ridge, HuberRegressor

In [3]:
from lightgbm import LGBMRegressor

In [4]:
import sys
sys.path.append('../')
from preprocessing.preprocessing import preprocessing

In [5]:
#gte = gte = pd.read_csv("../features/gte_features_w8_prp50.csv")
#gte.Date = pd.to_datetime(gte.Date)

In [35]:
path_train = '../dataset/prediction/val/'
path_test =  '../dataset/prediction/test/'

lgb_std_train = pd.read_csv(path_train + "lgb_std_val.csv")
lgb_std_test = pd.read_csv(path_test + "lgb_std_test.csv")

lgb_cls_train = pd.read_csv(path_train + "lgb_cls_val.csv")
lgb_cls_test = pd.read_csv(path_test + "lgb_cls_test.csv")

cat_std_train = pd.read_csv(path_train + "cat_std_val.csv")
cat_std_test = pd.read_csv(path_test + "cat_std_test.csv")

cat_cls_train = pd.read_csv(path_train + "cat_cls_val.csv")
cat_cls_test = pd.read_csv(path_test + "cat_cls_test.csv")

#xgb_train = pd.read_csv(path_train + "xgb_incremental_val.csv")
#xgb_test = pd.read_csv(path_test + "xgb_incremental_test.csv")

lin_reg_train = pd.read_csv(path_train + "linear_reg_val.csv")
lin_reg_test = pd.read_csv(path_test + "linear_reg_test.csv")

In [36]:
train = pd.read_csv("../dataset/original/train.csv")
test = pd.read_csv("../dataset/original/x_test.csv")
df = preprocessing(train, test, useTest=False)

In [37]:
prediction_train = lgb_cls_train.merge(lgb_std_train, how='left', on=['Date', 'sku', 'target', 'real_target'])
prediction_train = prediction_train.merge(cat_std_train, how='left', on=['Date', 'sku', 'target', 'real_target'])
prediction_train = prediction_train.merge(cat_cls_train, how='left', on=['Date', 'sku', 'target', 'real_target'])
#prediction_train = prediction_train.merge(xgb_train, how='left', on=['Date', 'sku', 'target', 'real_target'])
prediction_train = prediction_train.merge(lin_reg_train, how='left', on=['Date', 'sku', 'target', 'real_target'])

prediction_train.Date = pd.to_datetime(prediction_train.Date)
prediction_train = prediction_train.merge(df[['Date', 'sku', 'sales w-1']], how='left', on=['Date', 'sku'])
#prediction_train = prediction_train.merge(gte, how='left', on=['Date', 'sku', 'target', 'real_target'])

In [38]:
#prediction_train = prediction_train[prediction_train.log_prediction_lgb_cls != 0.0]

In [39]:
#prediction_train = prediction_train.merge(df[['Date', 'sku', 'sales w-1']], how='left')

In [41]:
prediction_test = lgb_cls_test.merge(lgb_std_test, how='left', on=['Date', 'sku', 'target', 'real_target'])
prediction_test = prediction_test.merge(cat_std_test, how='left', on=['Date', 'sku', 'target', 'real_target'])
prediction_test = prediction_test.merge(cat_cls_test, how='left', on=['Date', 'sku', 'target', 'real_target'])
#prediction_test = prediction_test.merge(xgb_test, how='left', on=['Date', 'sku', 'target', 'real_target'])
prediction_test = prediction_test.merge(lin_reg_test.drop(['target', 'real_target'], axis=1), how='left', on=['Date', 'sku'])

prediction_test.Date = pd.to_datetime(prediction_test.Date)
prediction_test = prediction_test.merge(df[['Date', 'sku', 'sales w-1']], how='left')
#prediction_test = prediction_test.merge(gte, how='left', on=['Date', 'sku', 'target', 'real_target'])

In [42]:
mask = (prediction_train.Date=='2017-01-07')
prediction_train = prediction_train.drop(prediction_train[mask].index)

In [43]:
prediction_test

,Date,sku,target,real_target,log_prediction_lgb_cls,prediction_lgb_cls,log_prediction_lgb_std,prediction_lgb_std,log_prediction_cat_std,prediction_cat_std,log_prediction_cat_cls,prediction_cat_cls,log_prediction_linear_reg,prediction_linear_reg,sales w-1
0,2019-06-29,144,9.694555,16228.0,9.519307,13619.166457,9.651065,15537.326992,9.728330,16785.503310,9.771958,17534.067699,9.960443,21171.167084,9.555348
1,2019-06-29,686,9.960718,21177.0,9.675532,15922.194960,9.776227,17609.086558,9.865336,19250.331824,9.966660,21303.212716,10.214279,27289.083189,9.808297
2,2019-06-29,1051,9.349145,11488.0,9.197323,9869.668955,9.286261,10787.765354,9.199808,9894.227945,9.258915,10496.740745,9.520431,13634.482152,9.046998
3,2019-06-29,546,11.025295,61407.0,11.013069,60660.769654,11.144993,69215.377494,11.044400,62591.443239,11.080064,64864.015467,11.033771,61929.675426,11.171547
4,2019-06-29,549,10.427565,33777.0,10.603837,40288.128989,10.584590,39520.086357,10.555070,38370.475778,10.528126,37350.393981,10.427925,33789.150922,10.663195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2019-12-14,1027,NaN,NaN,10.814699,49745.689993,10.805422,49286.332499,10.839962,51018.420675,10.771355,47635.513071,10.919550,55244.927711,10.770840
296,2019-12-14,1035,NaN,NaN,10.589340,39708.271172,10.619485,40923.534631,10.611096,40581.642897,10.621238,40995.318682,10.622325,41039.942721,10.547655
297,2019-12-14,1058,NaN,NaN,10.364807,31722.293645,10.316367,30222.251832,10.241657,28046.570818,10.345230,31107.311347,10.317320,30251.087468,10.232611
298,2019-12-14,1065,NaN,NaN,11.321390,82568.049283,11.425955,91669.404280,11.230041,75359.661675,11.292532,80219.335283,11.397453,89093.508610,11.282971


In [44]:
cols = ['log_prediction_lgb_cls',
        'log_prediction_lgb_std',
        'log_prediction_cat_std',
        'log_prediction_cat_cls',
        'log_prediction_linear_reg',
        #'prediction',
        #'sales w-1'
       ]
#gte_cols = [x for x in prediction_train.columns if 'gte' in x]
#cols = cols + gte_cols
cols

['log_prediction_lgb_cls',
 'log_prediction_lgb_std',
 'log_prediction_cat_std',
 'log_prediction_cat_cls',
 'log_prediction_linear_reg']

In [15]:
#prediction_train = prediction_train[prediction_train.Date > '2018-01-06']

In [45]:
reg = LinearRegression()
reg.fit(prediction_train[cols], prediction_train.target)

LinearRegression()

In [46]:
def stacking_gen(train, test):
    df = pd.concat([train, test])
    test_dates = test.Date.sort_values().drop_duplicates()
    for d in test_dates:
        yield df[df.Date < d], df[df.Date == d]

In [47]:
preds = pd.DataFrame()
for train, test in stacking_gen(prediction_train, prediction_test):
    reg = LinearRegression()
    reg.fit(train[cols], train.target)
    test['ensemble'] = reg.predict(test[cols]) / sum(reg.coef_)
    preds = pd.concat([preds, test])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [48]:
prediction_test['ensemble'] = reg.predict(prediction_test[cols]) / sum(reg.coef_)
prediction_test['ensemble'] = np.expm1(prediction_test.ensemble)

In [49]:
preds['ensemble'] = np.expm1(preds.ensemble)

In [50]:
from metrics.MAPE import MAPE

In [51]:
MAPE(preds.real_target, preds.ensemble)

nan

In [213]:
prediction_train[cols].columns

Index(['log_prediction_lgb_cls', 'log_prediction_lgb_std',
       'log_prediction_cat_std', 'log_prediction_cat_cls',
       'log_prediction_linear_reg'],
      dtype='object')

In [52]:
reg.coef_

array([0.04086052, 0.16208154, 0.03926392, 0.09368301, 0.66749698])

In [129]:
dates = prediction_train.Date.sort_values().drop_duplicates()

In [71]:
mapes = {}
for d in dates:
    mask = prediction_train.Date == d
    mapes[d] = MAPE(prediction_train[mask].real_target, prediction_train[mask].prediction_cat_std)
mapes

{Timestamp('2016-12-24 00:00:00'): 14.415419882614295,
 Timestamp('2016-12-31 00:00:00'): 45.562330082885005,
 Timestamp('2017-01-07 00:00:00'): 29.664381922703964,
 Timestamp('2017-01-14 00:00:00'): 27.541497877295473,
 Timestamp('2017-01-21 00:00:00'): 8.17772527419612,
 Timestamp('2017-01-28 00:00:00'): 6.301667653505562,
 Timestamp('2017-02-04 00:00:00'): 7.296266085058717,
 Timestamp('2017-02-11 00:00:00'): 10.796785933974483,
 Timestamp('2017-02-18 00:00:00'): 7.505157553437786,
 Timestamp('2017-02-25 00:00:00'): 4.86994972510147,
 Timestamp('2017-03-04 00:00:00'): 11.052292530335848,
 Timestamp('2017-03-11 00:00:00'): 8.44162649600763,
 Timestamp('2017-03-18 00:00:00'): 5.199012882941633,
 Timestamp('2017-03-25 00:00:00'): 17.919341595070225,
 Timestamp('2017-04-01 00:00:00'): 12.064057094624292,
 Timestamp('2017-04-08 00:00:00'): 10.196188944728227,
 Timestamp('2017-04-15 00:00:00'): 15.817711369351656,
 Timestamp('2017-04-22 00:00:00'): 14.63200076824852,
 Timestamp('2017-04-2

In [24]:
params = {'useTest':True, 'useScope':False}

In [16]:
def gen():
    a = [2,3,4,66,7]
    for i in a:
        yield i
        print('ciao')

In [21]:
for i in gen():
    print(i)

2
ciao
3
ciao
4
ciao
66
ciao
7
ciao


In [25]:
df

,Date,sku,pack,size (GM),brand,price,POS_exposed w-1,volume_on_promo w-1,sales w-1,scope,target,real_target
0,2016-12-10,144,0,114.23,1,2.18,NaN,NaN,10.497091,1,10.845855,51320.0
1,2016-12-17,144,0,114.23,1,2.00,45.0,100.000000,10.845855,1,11.103934,66431.0
2,2016-12-24,144,0,114.23,1,2.05,17.0,100.000000,11.103934,1,10.950842,57001.0
3,2016-12-31,144,0,114.23,1,3.00,2.0,100.000000,10.950842,1,9.619333,15052.0
4,2017-01-07,144,0,114.23,1,2.99,2.0,28.534193,9.619333,1,9.999570,22016.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6014,2019-05-25,2718,1,395.41,0,1.11,0.0,26.050480,10.430462,0,10.414183,33328.0
6015,2019-06-01,2718,1,395.41,0,1.30,1.0,43.099496,10.414183,0,10.021848,22512.0
6016,2019-06-08,2718,1,395.41,0,1.55,0.0,0.000000,10.021848,0,9.767782,17461.0
6017,2019-06-15,2718,1,395.41,0,1.55,0.0,0.000000,9.767782,0,9.747185,17105.0
